In [1]:
#include("defArc.jl")

type Arc
    initNode::Int 
    termNode::Int 
    capacity::Float64
    freeflowtime::Float64
    flow::Float64
end

Arc(initNode::Int, termNode::Int, capacity::Float64, freeflowtime::Float64) = 
    Arc(initNode, termNode, capacity, freeflowtime, 0.)

Arc

In [2]:
#include("fitTraffic.jl")

## Solve an inverse tarffic problem over polynomials 
## of degree at most d
## optionally use a regularizer from the poly kernel

using JuMP
using Gurobi
using Graphs


polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)])  

polyEval(coeffs::Array{Float64, 1}, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

bpacost(flow::Float64, capacity::Float64, freeflowtime::Float64) = freeflowtime*(1 + .15 * (flow/capacity)^4)
bpacost(flow::Float64, arc) = bpacost(flow, arc.capacity, arc.freeflowtime)
bpacost(arc::Arc) = bpacost(arc.flow, arc)

bpacost (generic function with 3 methods)

In [3]:
function setUpFitting(deg::Int, c::Float64)

	m = Model(solver=GurobiSolver(OutputFlag=false))
    
	@variable(m, coeffs[1:deg+1])
	@variable(m, Calphas[1:deg+1])

	#build the graham matrix; cf. Ref. [21] (Regularization Networks and Support Vector Machines), page 47
	samples = linspace(0, 1, deg + 1)
	k(x,y) = (c + x*y)^deg
	K = [ k(x,y) for x = samples, y=samples]
	K = convert(Array{Float64, 2}, K)
	#assert(rank(K) == deg+1)
    
	C = chol(K + 1e-6* eye(deg+1))
	for i=1:deg + 1
		@constraint(m, polyEval(coeffs, samples[i]) == sum{C[j, i] * Calphas[j], j=1:deg+1})
	end
    
	@variable(m, reg_term >= 0)
	reg_term_ = QuadExpr(Calphas[:], Calphas[:], ones(deg+1), AffExpr())
    
	@constraint(m, reg_term >= reg_term_)
    
	return m, coeffs, reg_term

end

setUpFitting (generic function with 1 method)

In [4]:
function fixCoeffs(m, fcoeffs, coeffs)
	for (fc, c) in zip(fcoeffs, coeffs[:])
		@constraint(m, fc == c)
	end
end

fixCoeffs (generic function with 1 method)

In [5]:
function addResid(m, coeffs, ys, demands, arcs, scaling)
	@variable(m, resid)
	@variable(m, dual_cost)
	@variable(m, primal_cost)

	@constraint(m, dual_cost == sum{demands[(s,t)] * (ys[(s,t), t] - ys[(s,t), s]), (s,t)=keys(demands)})  
	@constraint(m, primal_cost == sum{a.flow * a.freeflowtime * polyEval(coeffs, a.flow/a.capacity), a=values(arcs)})

	@constraint(m, resid >= (dual_cost - primal_cost) / scaling )
	@constraint(m, resid >= (primal_cost - dual_cost) / scaling )
	return resid
end

addResid (generic function with 1 method)

In [6]:
function addIncreasingCnsts(m, coeffs, arcs; TOL=0.)
	sorted_flows = sort([a.flow / a.capacity for a in values(arcs)])
	@constraint(m, polyEval(coeffs, 0) <= polyEval(coeffs, sorted_flows[1]))
	for i = 2:length(sorted_flows)
		@constraint(m, polyEval(coeffs, sorted_flows[i-1]) <= polyEval(coeffs, sorted_flows[i]) + TOL)
	end
    @constraint(m, coeffs[1] == 1)
end

addIncreasingCnsts (generic function with 1 method)

In [7]:
#equates the total cost of the network to the true total cost
function normalize(m, coeffs, tot_true_cost::Float64, arcs)
	@constraint(m, 
		sum{a.freeflowtime * a.flow * polyEval(coeffs, a.flow / a.capacity), a=values(arcs)} == tot_true_cost)
end

function normalize(m, coeffs, scaled_flow::Float64, cost::Float64)
	@constraint(m, polyEval(coeffs, scaled_flow) == cost)
end

function normalize(m, coeffs, scaled_flows::Array{Float64, 1}, avgCost::Float64)
    @constraint(m, sum{polyEval(coeffs, f), f=scaled_flows} == avgCost * length(scaled_flows))
end

normalize (generic function with 3 methods)

In [8]:
function addNetworkCnsts(m, coeffs, demands, arcs, numNodes)
	@variable(m, ys[keys(demands), 1:numNodes])
	for k = keys(arcs)
		a = arcs[k]
		rhs = a.freeflowtime * polyEval(coeffs, a.flow/a.capacity)
		for od in keys(demands)
			@constraint(m, ys[od, k[2]] - ys[od, k[1]] <= rhs)
		end
	end
	return ys
end

addNetworkCnsts (generic function with 1 method)

In [9]:
############
#Read in demand data
file = open("../temp_files/OD_demand_matrix_Apr_weekend.txt")
demands = Dict()
n = 8  # number of nodes
for i = 1:n
    demands[(i,i)] = 0.0
end
for line in eachline(file)
    OD_demand = split(line, ",")
    key, value = (parse(Int, OD_demand[1]), parse(Int, OD_demand[2])), parse(Float64, split(OD_demand[3], "\n")[1])
    demands[key] = value
end
close(file)

In [10]:
demands

Dict{Any,Any} with 64 entries:
  (3,6) => 492.255
  (4,4) => 0.0
  (3,1) => 550.454
  (4,5) => 297.82
  (2,4) => 262.686
  (6,5) => 337.643
  (8,4) => 393.75
  (1,2) => 469.814
  (5,1) => 357.213
  (3,4) => 337.59
  (8,2) => 204.438
  (2,3) => 370.061
  (2,6) => 435.446
  (5,4) => 315.592
  (2,1) => 497.169
  (2,5) => 416.029
  (8,5) => 302.149
  (1,4) => 216.96
  (2,8) => 280.07
  (5,8) => 277.482
  (4,8) => 207.333
  (7,8) => 272.515
  (1,1) => 0.0
  (7,7) => 0.0
  (5,7) => 486.899
  ⋮     => ⋮

In [11]:
using JSON

link_day_minute_Apr_dict = readstring("../temp_files/link_day_minute_Apr_dict_JSON_adjusted.json");
link_day_minute_Apr_dict = replace(link_day_minute_Apr_dict, "NaN", 0);

link_day_minute_Apr_dict = JSON.parse(link_day_minute_Apr_dict);

In [12]:
# weekend_Apr_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

# training set 1
weekend_Apr_list_1 = [1, 7, 8, 14, 15, 21]

# training set 2
weekend_Apr_list_2 = [14, 15, 21, 22, 28, 29]

# training set 3
weekend_Apr_list_3 = [1, 7, 8, 22, 28, 29]

arcs_1 = Dict[]
for j in weekend_Apr_list_1
    arcs_1_ = Dict()
    for i = 0:23
        key = "link_$(i)_$(j)"
        initNode = link_day_minute_Apr_dict[key]["init_node"]
        termNode = link_day_minute_Apr_dict[key]["term_node"]
        capacity = link_day_minute_Apr_dict[key]["MD_capac"]
        freeflowtime = link_day_minute_Apr_dict[key]["free_flow_time"]
        flow = link_day_minute_Apr_dict[key]["MD_flow"]
        arcs_1_[(initNode, termNode)] = Arc(initNode, termNode, capacity, freeflowtime, flow)
    end
    push!(arcs_1, arcs_1_)
end

arcs_2 = Dict[]
for j in weekend_Apr_list_2
    arcs_2_ = Dict()
    for i = 0:23
        key = "link_$(i)_$(j)"
        initNode = link_day_minute_Apr_dict[key]["init_node"]
        termNode = link_day_minute_Apr_dict[key]["term_node"]
        capacity = link_day_minute_Apr_dict[key]["MD_capac"]
        freeflowtime = link_day_minute_Apr_dict[key]["free_flow_time"]
        flow = link_day_minute_Apr_dict[key]["MD_flow"]
        arcs_2_[(initNode, termNode)] = Arc(initNode, termNode, capacity, freeflowtime, flow)
    end
    push!(arcs_2, arcs_2_)
end

arcs_3 = Dict[]
for j in weekend_Apr_list_3
    arcs_3_ = Dict()
    for i = 0:23
        key = "link_$(i)_$(j)"
        initNode = link_day_minute_Apr_dict[key]["init_node"]
        termNode = link_day_minute_Apr_dict[key]["term_node"]
        capacity = link_day_minute_Apr_dict[key]["MD_capac"]
        freeflowtime = link_day_minute_Apr_dict[key]["free_flow_time"]
        flow = link_day_minute_Apr_dict[key]["MD_flow"]
        arcs_3_[(initNode, termNode)] = Arc(initNode, termNode, capacity, freeflowtime, flow)
    end
    push!(arcs_3, arcs_3_)
end

In [13]:
##########
# Set up demand data and flow data
##########
numData = length(arcs_1); 
sigma = .0

flow_data_1 = Array(Float64, length(arcs_1[1]), numData)

demand_data = Dict()

numNodes = maximum(map(pair->pair[1], keys(demands)))
g = simple_inclist(numNodes, is_directed=true)
vArcs = Arc[]
for arc in values(arcs_1[1])
    add_edge!(g, arc.initNode, arc.termNode) 
    push!(vArcs, arc)
end


for iRun = 1:numData
    for odpair in keys(demands)
        if ! haskey(demand_data, odpair)
            demand_data[odpair] = [demands[odpair], ]
        else
            push!(demand_data[odpair], demands[odpair])
        end
    end

    #flow_data[:, iRun] = [a.flow::Float64 for a in vArcs]
    flow_data_1[:, iRun] = [a.flow::Float64 for a in values(arcs_1[iRun])]
end

In [14]:
##########
# Set up demand data and flow data
##########
numData = length(arcs_2); 
sigma = .0

flow_data_2 = Array(Float64, length(arcs_2[1]), numData)

demand_data = Dict()

numNodes = maximum(map(pair->pair[1], keys(demands)))
g = simple_inclist(numNodes, is_directed=true)
vArcs = Arc[]
for arc in values(arcs_2[1])
    add_edge!(g, arc.initNode, arc.termNode) 
    push!(vArcs, arc)
end


for iRun = 1:numData
    for odpair in keys(demands)
        if ! haskey(demand_data, odpair)
            demand_data[odpair] = [demands[odpair], ]
        else
            push!(demand_data[odpair], demands[odpair])
        end
    end

    #flow_data[:, iRun] = [a.flow::Float64 for a in vArcs]
    flow_data_2[:, iRun] = [a.flow::Float64 for a in values(arcs_2[iRun])]
end

In [15]:
##########
# Set up demand data and flow data
##########
numData = length(arcs_3); 
sigma = .0

flow_data_3 = Array(Float64, length(arcs_3[1]), numData)

demand_data = Dict()

numNodes = maximum(map(pair->pair[1], keys(demands)))
g = simple_inclist(numNodes, is_directed=true)
vArcs = Arc[]
for arc in values(arcs_3[1])
    add_edge!(g, arc.initNode, arc.termNode) 
    push!(vArcs, arc)
end


for iRun = 1:numData
    for odpair in keys(demands)
        if ! haskey(demand_data, odpair)
            demand_data[odpair] = [demands[odpair], ]
        else
            push!(demand_data[odpair], demands[odpair])
        end
    end

    #flow_data[:, iRun] = [a.flow::Float64 for a in vArcs]
    flow_data_3[:, iRun] = [a.flow::Float64 for a in values(arcs_3[iRun])]
end

In [16]:
demand_data

Dict{Any,Any} with 64 entries:
  (3,6) => [492.255,492.255,492.255,492.255,492.255,492.255]
  (4,4) => [0.0,0.0,0.0,0.0,0.0,0.0]
  (3,1) => [550.454,550.454,550.454,550.454,550.454,550.454]
  (4,5) => [297.82,297.82,297.82,297.82,297.82,297.82]
  (2,4) => [262.686,262.686,262.686,262.686,262.686,262.686]
  (6,5) => [337.643,337.643,337.643,337.643,337.643,337.643]
  (8,4) => [393.75,393.75,393.75,393.75,393.75,393.75]
  (1,2) => [469.814,469.814,469.814,469.814,469.814,469.814]
  (5,1) => [357.213,357.213,357.213,357.213,357.213,357.213]
  (3,4) => [337.59,337.59,337.59,337.59,337.59,337.59]
  (8,2) => [204.438,204.438,204.438,204.438,204.438,204.438]
  (2,3) => [370.061,370.061,370.061,370.061,370.061,370.061]
  (2,6) => [435.446,435.446,435.446,435.446,435.446,435.446]
  (5,4) => [315.592,315.592,315.592,315.592,315.592,315.592]
  (2,1) => [497.169,497.169,497.169,497.169,497.169,497.169]
  (2,5) => [416.029,416.029,416.029,416.029,416.029,416.029]
  (8,5) => [302.149,302.149,302.149

In [17]:
flow_data_3

24×6 Array{Float64,2}:
 2810.46  2490.09  2653.07  2623.14  2575.26  2072.79
 2555.64  2297.57  2826.53  2562.92  2115.27  2228.51
 1475.03  1497.18  1691.3   1456.37  1416.8   1554.31
 1637.71  1534.33  1906.74  1987.36  1320.83  1763.26
 1992.23  1787.07  2080.61  1951.55  1842.33  1959.01
 2940.31  2496.24  2988.73  2338.26  2461.37  2303.19
 2622.65  2183.5   3016.4   2785.14  1963.08  2407.62
 2541.89  2014.83  2747.75  2384.89  2108.08  2228.88
 1785.79  1173.28  2145.84  1730.56  1179.12  1999.87
 1976.58  1615.49  2269.54  2157.9   1187.59  1745.6 
 2192.52  2385.01  2440.88  2506.02  1628.62  2208.46
 2185.5   1711.88  2375.34  2358.36  1476.26  1891.35
 2536.74  1914.94  2628.1   2091.29  1761.11  2277.02
 1637.05  1466.4   1653.35   878.48  1251.69  1275.28
 2132.47  1691.2   2386.37  2159.22  1386.71  1973.5 
 2271.71  1564.78  2544.24  2172.94  1833.7   1900.71
 1486.95  1365.92  1277.27  1182.38  1367.16  1163.93
 1495.28  1591.51  1731.72  1437.37  1415.23  1435.26
 2062

In [18]:
##########
#Fitting Funcs
##########

function train(indices, lam::Float64, deg::Int, c::Float64, demand_data, flow_data, arcs; fcoeffs=nothing)
    numNodes = maximum(map(pair->pair[1], keys(arcs)))
    m, coeffs, reg_term = setUpFitting(deg, c)
    
    addIncreasingCnsts(m, coeffs, arcs, TOL=1e-8)  #uses the original obs flows

    avgCost = mean( [bpacost(a.flow, a.capacity, 1.0) for a in values(arcs)] )
    normalize(m, coeffs, [a.flow / a.capacity for a in values(arcs)], avgCost)


    resids = Variable[]
    
    for i = indices
        #copy the flow data over to the arcs, demand data to demands (slow)
        for (ix, a) in enumerate(vArcs)
            a.flow = flow_data[ix, i]
        end
        for odpair in keys(demands)
            demands[odpair] = demand_data[odpair][i]
        end
    
        #Dual Feasibility
        ys = addNetworkCnsts(m, coeffs, demands, arcs, numNodes)
        
        #add the residual for this data point
        push!(resids, addResid(m, coeffs, ys, demands, arcs, 1e6))
    end

    if fcoeffs != nothing
        fixCoeffs(m, fcoeffs, coeffs)
    end
    @objective(m, Min, sum{resids[i], i = 1:length(resids)} + lam*reg_term)
    solve(m)
    #println(getObjectiveValue(m) - lam * getValue(reg_term) )
    return [getValue(coeffs[i]) for i =1:length(coeffs)]
end

train (generic function with 1 method)

In [19]:
#include("trafficCval.jl")

coeffs_dict_Apr_weekend = Dict()

deg_grid = 4:8
c_grid = .5:.5:3.
lamb_grid = 10. .^(-3:4)

for deg in deg_grid
    for c in c_grid
        for lam in lamb_grid
            coeffs_dict_Apr_weekend[(deg, c, lam, 1)] = train(1:numData, lam, deg, c, demand_data, flow_data_1, arcs_1[1])
            coeffs_dict_Apr_weekend[(deg, c, lam, 2)] = train(1:numData, lam, deg, c, demand_data, flow_data_2, arcs_2[1])
            coeffs_dict_Apr_weekend[(deg, c, lam, 3)] = train(1:numData, lam, deg, c, demand_data, flow_data_3, arcs_3[1])
        end
    end
end

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in getValue(::JuMP.Variable, ::Vararg{JuMP.Variable,N}) at ./deprecated.jl:30
 in collect(::Base.Generator{UnitRange{Int64},##27#31}) at ./array.jl:307
 in #train#23(::Void, ::Function, ::UnitRange{Int64}, ::Float64, ::Int64, ::Float64, ::Dict{Any,Any}, ::Array{Float64,2}, ::Dict{Any,Any}) at ./In[18]:39
 in train(::UnitRange{Int64}, ::Float64, ::Int64, ::Float64, ::Dict{Any,Any}, ::Array{Float64,2}, ::Dict{Any,Any}) at ./In[18]:6
 in macro expansion; at ./In[19]:12 [inlined]
 in anonymous at ./<missing>:?
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/jzh/.julia/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/jzh/.julia/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading In[19], in expression starting on line 9


In [20]:
outfile = open("../temp_files/coeffs_dict_Apr_weekend.json", "w")

JSON.print(outfile, coeffs_dict_Apr_weekend)

close(outfile)

In [21]:
coeffs_dict_Apr_weekend_ = readstring("../temp_files/coeffs_dict_Apr_weekend.json")
coeffs_dict_Apr_weekend_ = JSON.parse(coeffs_dict_Apr_weekend_)

Dict{String,Any} with 720 entries:
  "(7,2.5,10000.0,3)" => Any[1.0,0.00405683,0.0014504,0.000289832,3.67226e-5,7.…
  "(5,0.5,100.0,3)"   => Any[1.0,0.00282311,0.00539145,-0.000417158,0.00328862,…
  "(6,1.5,10.0,3)"    => Any[1.0,0.00419926,0.0011512,-0.000149134,-0.000227726…
  "(6,2.0,0.1,2)"     => Any[1.0,0.00654626,-0.0132375,0.00661325,0.00236065,0.…
  "(4,0.5,1000.0,1)"  => Any[1.0,0.00444865,0.00510865,0.00178841,0.000565036]
  "(6,2.0,0.01,1)"    => Any[1.0,-0.00754739,0.0229389,0.0462696,0.0112612,0.00…
  "(8,1.5,0.01,1)"    => Any[1.0,-0.00103714,-0.0180105,0.082241,0.0886698,-0.0…
  "(5,1.5,1.0,1)"     => Any[1.0,0.00532673,0.00283115,0.00114905,0.000179329,5…
  "(8,2.5,0.001,2)"   => Any[1.0,0.0123963,-0.0623888,0.144516,-0.158899,0.1269…
  "(4,2.0,0.1,1)"     => Any[1.0,0.00535863,0.00291904,0.000681314,4.86456e-5]
  "(5,1.0,1000.0,2)"  => Any[1.0,0.00298538,0.00166395,0.000358955,0.000141913,…
  "(4,2.0,0.1,2)"     => Any[1.0,0.00542285,-0.00677766,-0.000998521,-3.13786e

In [22]:
coeffs_dict_Apr_weekend_["(6,0.5,10000.0,1)"]

7-element Array{Any,1}:
  1.0       
  0.00268854
  0.0114665 
 -0.00834402
  0.0234204 
 -0.0137302 
  0.00429397

In [23]:
using PyPlot

fcoeffs = coeffs_dict_Apr_weekend_["(6,0.5,10000.0,1)"]


xs = linspace(0, 1.1, 30)
ys = map(x->polyEval(fcoeffs, x), xs)

plot(xs, ys, "o-m", label="Apr_weekend")

#legend(loc="upper left",fancybox="true") 

grid("on")
xlim(-0.1, 1.05);
ylim(0.99, 1.2);

font1 = Dict("family"=>"serif","color"=>"k","weight"=>"normal","size"=>14)

xlabel("Scaled Flow", fontdict=font1)

savefig("../temp_files/fitting_Apr_weekend.pdf")

LoadError: LoadError: UndefVarError: plot not defined
while loading In[23], in expression starting on line 9